In [ ]:
import csv
import re
from gensim.models import Word2Vec
import pandas as pd 
import numpy as np
from sklearn.datasets import load_files
from collections import Counter 
from nltk.tokenize import word_tokenize
import gc
from nltk.stem.snowball import SnowballStemmer
 
from joblib import Parallel, delayed
import multiprocessing
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

# Import Data

In [ ]:
def majid(X,Punc=False):
    corpus = []
    for i in range(0, len(X)):
        if Punc:
          review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(X[i])) #remove punctuation
        review = re.sub(r'\d+',' ', str(X[i]))# remove number
        review = review.lower() #lower case
        review = re.sub(r'\s+', ' ', review) #remove extra space
        review = re.sub(r'<[^>]+>','',review) #remove Html tags
        review = re.sub(r'\s+', ' ', review) #remove spaces
        review = re.sub(r"^\s+", '', review) #remove space from start
        review = re.sub(r'\s+$', '', review) #remove space from the end
        corpus.append(review)             
    #Tokenizing and Word Count  
    words=[]
    for i in range(len(corpus)):
        words= nltk.word_tokenize(corpus[i])
        #sentences.append(words)
    return words

In [ ]:
path = "../data/articles1.csv"
News = pd.read_csv(path)
text= News['content']
X=text.values.tolist()
X = [[el] for el in X] 
num_cores = multiprocessing.cpu_count()
sentences = Parallel(n_jobs=num_cores)(delayed(majid)(i) for i in X)

In [ ]:
def majid2(X,func):
    review = func(X)
    gc.collect()
    return review

### Post Tagging

In [ ]:
def pos_tagger(sentences):
    tags = [] #have the pos tag included
    nava_sen = []
    pt = nltk.pos_tag(sentences)
    nava = []
    nava_words = []
    for t in pt:
        if t[1].startswith('NN') or t[1].startswith('NNS') or t[1].startswith('NNP') or t[1].startswith('NNPS') or t[1].startswith('JJ') or t[1].startswith('JJR') or t[1].startswith('JJS') or  t[1].startswith('VB') or t[1].startswith('VBG') or t[1].startswith('VBN') or t[1].startswith('VBP') or t[1].startswith('VBZ') or t[1].startswith('RB') or t[1].startswith('RBR') or t[1].startswith('RBS'):
            nava.append(t)
            nava_words.append(t[0])
    return nava_words

In [ ]:
from tqdm import tqdm
num_cores = multiprocessing.cpu_count()
sent2 = Parallel(n_jobs=num_cores)(delayed(majid2)(i,pos_tagger) for i in tqdm(sentences))
count = 0
c = {}
for words in sent2:
  for s in words:
    if s in c:
        c[s] += 1
    else:
        c[s] = 1
    count+=1
d= []

for k,v in c.items():
    if v == 1:
        d.append(k) 
    
print('Corpus Size=', count)        
print('Unique words=', len(d)) 

100%|██████████| 50000/50000 [43:48<00:00, 19.02it/s]


Corpus Size= 21854171
Unique words= 69540


In [ ]:

"""""
Making Vocabulary and Training the Model
(sg=0 CBOW , sg=1 Skip-gram)
"""""
#########
#path = "/content/drive/MyDrive/NLP/embeddings/"
CBOW = Word2Vec(sentences, min_count=3,size= 300,workers=multiprocessing.cpu_count(), window =1, sg = 0)
print('Done Training')

SizeOfVocab = CBOW.wv.vocab
print('Size of Vocabulary=',len(SizeOfVocab))
print('Done making the Vocabulary')

#####
Skip_gram = Word2Vec(sentences, min_count=3,size= 300,workers=multiprocessing.cpu_count(), window =1, sg = 1)
print('Done Training')

SizeOfVocab = Skip_gram.wv.vocab
print('Size of Vocabulary=',len(SizeOfVocab))
print('Done making the Vocabulary')

""""
Saving the embeddings and the model
"""""

from gensim.models import Word2Vec, KeyedVectors   
#CBOW.wv.save_word2vec_format('W-CBOW-POS.bin.gz', binary=True)
path = "../Embeddings/"
CBOW.wv.save_word2vec_format(path + "W-CBOW-POS.txt", binary=False)
CBOW.save('W-CBOW-POS.bin')
print('Done Saving CBOW')
#####
#Skip_gram.wv.save_word2vec_format('W-Skip-POS.gz', binary=True)
path = "../Embeddings/"
Skip_gram.wv.save_word2vec_format(path + "W-Skip-POS.txt", binary=False)
Skip_gram.save('W-Skip-POS.bin')
print('Done Saving Skip_gram')
print('Done Saving the Embeddings')

Done Training
Size of Vocabulary= 88803
Done making the Vocabulary
Done Training
Size of Vocabulary= 88803
Done making the Vocabulary
Done Saving CBOW
Done Saving Skip_gram
Done Saving the Embeddings
